# Dropped because of too HIGH RM required

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import logging
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import gc 

pd.set_option('display.max_columns', None)
sns.set_theme()  
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 8]

sys.path.append('../src')
from preprocessing import MIMICPreprocessor

In [2]:
DATA_PATH = Path('../data')
PROCESSED_PATH = Path('../data/processed')
PROCESSED_PATH.mkdir(exist_ok=True)
RANDOM_SEED = 42

preprocessor = MIMICPreprocessor(DATA_PATH)

In [3]:
# 第1步：加载和处理 PATIENTS 表（较小的表）
print("Step 1: Processing PATIENTS table...")
try:
    patients_df = pd.read_csv(
        DATA_PATH / "PATIENTS.csv",
        usecols=['SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG'],
        dtype={
            'SUBJECT_ID': 'int32',
            'GENDER': 'category',
            'EXPIRE_FLAG': 'int8'
        }
    )
    patients_df = preprocessor.process_demographics(patients_df)
    print(f"Processed {len(patients_df)} patients")
    
    # 保存处理后的患者数据
    patients_df.to_pickle(PROCESSED_PATH / 'processed_patients.pkl')
    gc.collect()  # 清理内存
except Exception as e:
    print(f"Error processing PATIENTS: {str(e)}")
    raise

2024-11-16 03:25:52,893 - INFO - Processing demographics data...


Step 1: Processing PATIENTS table...


2024-11-16 03:25:54,704 - INFO - 
Demographic Processing Summary:
2024-11-16 03:25:54,704 - INFO - Total patients: 46520
2024-11-16 03:25:54,705 - INFO - Patients with accurate age (≤89): 28386
2024-11-16 03:25:54,706 - INFO - Patients with age >89 (marked as 90+): 18134


Processed 46520 patients


In [4]:
# 第2步：分块处理 NOTEEVENTS 表
print("\nStep 2: Processing NOTEEVENTS table in chunks...")
try:
    chunk_size = 50000  # 可以根据内存调整
    notes_chunks = []
    
    for chunk in tqdm(pd.read_csv(
        DATA_PATH / "NOTEEVENTS.csv",
        usecols=['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CATEGORY', 'TEXT'],
        chunksize=chunk_size
    )):
        processed_chunk = preprocessor.clean_notes(chunk)
        notes_chunks.append(processed_chunk[['SUBJECT_ID', 'HADM_ID', 'cleaned_text', 'sections']])
        del chunk
        gc.collect()

    notes_df = pd.concat(notes_chunks, ignore_index=True)
    del notes_chunks
    gc.collect()
    
    # 保存处理后的临床记录
    notes_df.to_pickle(PROCESSED_PATH / 'processed_notes.pkl')
    print(f"Processed {len(notes_df)} clinical notes")
except Exception as e:
    print(f"Error processing NOTEEVENTS: {str(e)}")
    raise



Step 2: Processing NOTEEVENTS table in chunks...


0it [00:00, ?it/s]2024-11-16 03:26:01,218 - INFO - Starting clinical notes cleaning...

Cleaning text: 0it [00:00, ?it/s]

Extracting sections: 0it [00:00, ?it/s]
1it [00:03,  3.11s/it]2024-11-16 03:26:02,447 - INFO - Starting clinical notes cleaning...

Cleaning text: 0it [00:00, ?it/s]

Extracting sections: 0it [00:00, ?it/s]
2it [00:04,  2.00s/it]2024-11-16 03:26:02,698 - INFO - Starting clinical notes cleaning...

Cleaning text: 0it [00:00, ?it/s]

Extracting sections: 0it [00:00, ?it/s]
3it [00:04,  1.20s/it]2024-11-16 03:26:02,852 - INFO - Starting clinical notes cleaning...

Cleaning text: 0it [00:00, ?it/s]

Extracting sections: 0it [00:00, ?it/s]
4it [00:04,  1.27it/s]2024-11-16 03:26:03,003 - INFO - Starting clinical notes cleaning...

Cleaning text: 0it [00:00, ?it/s]

Extracting sections: 0it [00:00, ?it/s]
5it [00:04,  1.79it/s]2024-11-16 03:26:03,157 - INFO - Starting clinical notes cleaning...

Cleaning text: 0it [00:00, ?it/s]

Extracting sections: 0it [00:00, ?it/s]
6i

Cleaning text: 100%|█████████████████████████████████████████████| 17609/17609 [00:05<00:00, 3370.36it/s]
../src/preprocessing.py:316: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df['cleaned_text'] = notes_df['TEXT'].progress_apply(clean_text)

Extracting sections: 100%|███████████████████████████████████████| 17609/17609 [00:07<00:00, 2418.29it/s]
../src/preprocessing.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df['sections'] = notes_df['cleaned_text'].progress_apply(extract_section

Extracting sections: 100%|███████████████████████████████████████| 16169/16169 [00:06<00:00, 2530.85it/s]
../src/preprocessing.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df['sections'] = notes_df['cleaned_text'].progress_apply(extract_sections)
9it [00:40,  8.43s/it]2024-11-16 03:26:39,829 - INFO - Starting clinical notes cleaning...

Cleaning text: 100%|█████████████████████████████████████████████| 15894/15894 [00:04<00:00, 3324.44it/s]
../src/preprocessing.py:316: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

Extracting sections: 100%|███████████████████████████████████████| 15894/15894 [00:06<00:00, 2436.16it/s]
../src/preprocessing.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df['sections'] = notes_df['cleaned_text'].progress_apply(extract_sections)
10it [00:53,  9.66s/it]2024-11-16 03:26:52,255 - INFO - Starting clinical notes cleaning...

Cleaning text: 100%|█████████████████████████████████████████████| 15553/15553 [00:04<00:00, 3361.94it/s]
../src/preprocessing.py:316: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

Cleaning text: 100%|█████████████████████████████████████████████| 16244/16244 [00:04<00:00, 3331.34it/s]
../src/preprocessing.py:316: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df['cleaned_text'] = notes_df['TEXT'].progress_apply(clean_text)

Extracting sections: 100%|███████████████████████████████████████| 16244/16244 [00:06<00:00, 2489.53it/s]
../src/preprocessing.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df['sections'] = notes_df['cleaned_text'].progress_apply(extract_section

Cleaning text: 100%|█████████████████████████████████████████████| 15384/15384 [00:04<00:00, 3296.68it/s]
../src/preprocessing.py:316: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df['cleaned_text'] = notes_df['TEXT'].progress_apply(clean_text)

Extracting sections: 100%|███████████████████████████████████████| 15384/15384 [00:06<00:00, 2429.29it/s]
../src/preprocessing.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df['sections'] = notes_df['cleaned_text'].progress_apply(extract_section

13it [01:29, 11.37s/it]2024-11-16 03:27:28,946 - INFO - Starting clinical notes cleaning...

Cleaning text: 100%|█████████████████████████████████████████████| 16426/16426 [00:04<00:00, 3377.40it/s]
../src/preprocessing.py:316: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df['cleaned_text'] = notes_df['TEXT'].progress_apply(clean_text)

Extracting sections: 100%|███████████████████████████████████████| 16426/16426 [00:06<00:00, 2523.06it/s]
../src/preprocessing.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

Cleaning text: 100%|█████████████████████████████████████████████| 10606/10606 [00:03<00:00, 3233.43it/s]
../src/preprocessing.py:316: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df['cleaned_text'] = notes_df['TEXT'].progress_apply(clean_text)

Extracting sections: 100%|███████████████████████████████████████| 10606/10606 [00:04<00:00, 2382.43it/s]
../src/preprocessing.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df['sections'] = notes_df['cleaned_text'].progress_apply(extract_section

Processed 141624 clinical notes


In [5]:
# 第3步：分块处理 PRESCRIPTIONS 表
print("\nStep 3: Processing PRESCRIPTIONS table in chunks...")
try:
    chunk_size = 100000  # 可以根据内存调整
    prescription_chunks = []
    
    for chunk in tqdm(pd.read_csv(
        DATA_PATH / "PRESCRIPTIONS.csv",
        usecols=['SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG', 'ROUTE', 
                 'DOSE_VAL_RX', 'DOSE_UNIT_RX'],
        dtype={
            'SUBJECT_ID': 'int32',
            'HADM_ID': 'int32',
            'DRUG': 'category',
            'ROUTE': 'category',
            'DOSE_UNIT_RX': 'category'
        },
        chunksize=chunk_size
    )):
        processed_chunk = preprocessor.process_prescriptions(chunk)
        prescription_chunks.append(processed_chunk)
        del chunk
        gc.collect()

    prescriptions_df = pd.concat(prescription_chunks, ignore_index=True)
    del prescription_chunks
    gc.collect()
    
    # 保存处理后的处方数据
    prescriptions_df.to_pickle(PROCESSED_PATH / 'processed_prescriptions.pkl')
    print(f"Processed {len(prescriptions_df)} prescriptions")
except Exception as e:
    print(f"Error processing PRESCRIPTIONS: {str(e)}")
    raise


Step 3: Processing PRESCRIPTIONS table in chunks...


0it [00:00, ?it/s]2024-11-16 03:29:25,709 - INFO - Processing prescription data...
2024-11-16 03:29:29,962 - INFO - Processed 100000 prescriptions
2024-11-16 03:29:29,963 - INFO - Available columns after prescription processing:
2024-11-16 03:29:29,964 - INFO - ['SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'ROUTE', 'DURATION', 'DRUG_NAME_GENERIC', 'drug_id']
1it [00:04,  4.42s/it]2024-11-16 03:29:30,120 - INFO - Processing prescription data...
2024-11-16 03:29:34,365 - INFO - Processed 100000 prescriptions
2024-11-16 03:29:34,365 - INFO - Available columns after prescription processing:
2024-11-16 03:29:34,366 - INFO - ['SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'ROUTE', 'DURATION', 'DRUG_NAME_GENERIC', 'drug_id']
2it [00:08,  4.41s/it]2024-11-16 03:29:34,524 - INFO - Processing prescription data...
2024-11-16 03:29:38,777 - INFO - Processed 100000 prescriptions
2024-11-16 03:29:38,778 - INFO - Avai

2024-11-16 03:30:58,447 - INFO - Processed 100000 prescriptions
2024-11-16 03:30:58,448 - INFO - Available columns after prescription processing:
2024-11-16 03:30:58,448 - INFO - ['SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'ROUTE', 'DURATION', 'DRUG_NAME_GENERIC', 'drug_id']
21it [01:32,  4.43s/it]2024-11-16 03:30:58,607 - INFO - Processing prescription data...
2024-11-16 03:31:02,891 - INFO - Processed 100000 prescriptions
2024-11-16 03:31:02,892 - INFO - Available columns after prescription processing:
2024-11-16 03:31:02,893 - INFO - ['SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'ROUTE', 'DURATION', 'DRUG_NAME_GENERIC', 'drug_id']
22it [01:37,  4.44s/it]2024-11-16 03:31:03,052 - INFO - Processing prescription data...
2024-11-16 03:31:07,325 - INFO - Processed 100000 prescriptions
2024-11-16 03:31:07,326 - INFO - Available columns after prescription processing:
2024-11-16 03:31:07,327 - INFO - ['S

2024-11-16 03:32:27,194 - INFO - Available columns after prescription processing:
2024-11-16 03:32:27,195 - INFO - ['SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'ROUTE', 'DURATION', 'DRUG_NAME_GENERIC', 'drug_id']
41it [03:01,  4.44s/it]2024-11-16 03:32:27,308 - INFO - Processing prescription data...
2024-11-16 03:32:29,696 - INFO - Processed 56450 prescriptions
2024-11-16 03:32:29,697 - INFO - Available columns after prescription processing:
2024-11-16 03:32:29,697 - INFO - ['SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'ROUTE', 'DURATION', 'DRUG_NAME_GENERIC', 'drug_id']
42it [03:04,  4.38s/it]


Processed 4156450 prescriptions


In [6]:
# 第4步：处理 DIAGNOSES_ICD 表
print("\nStep 4: Processing DIAGNOSES_ICD table...")
try:
    diagnoses_df = pd.read_csv(
        DATA_PATH / "DIAGNOSES_ICD.csv",
        usecols=['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE'],
        dtype={
            'SUBJECT_ID': 'int32',
            'HADM_ID': 'int32',
            'ICD9_CODE': 'category'
        }
    )
    diagnoses_df.to_pickle(PROCESSED_PATH / 'processed_diagnoses.pkl')
    print(f"Processed {len(diagnoses_df)} diagnoses")
except Exception as e:
    print(f"Error processing DIAGNOSES_ICD: {str(e)}")
    raise


Step 4: Processing DIAGNOSES_ICD table...
Processed 651047 diagnoses


In [7]:
# 第5步：分析和可视化（可选，根据需要运行）
def analyze_processed_data():
    print("\nAnalyzing processed data...")
    
    # 加载处理后的数据
    patients_df = pd.read_pickle(PROCESSED_PATH / 'processed_patients.pkl')
    
    # 绘制人口统计图
    plot_demographics(patients_df)
    
    # 分析处方数据（使用样本以节省内存）
    prescriptions_df = pd.read_pickle(PROCESSED_PATH / 'processed_prescriptions.pkl')
    prescription_sample = prescriptions_df.sample(n=min(100000, len(prescriptions_df)))
    plot_prescriptions_summary(prescription_sample)
    del prescription_sample
    gc.collect()
    
    # 分析临床记录
    notes_df = pd.read_pickle(PROCESSED_PATH / 'processed_notes.pkl')
    analyze_notes_summary(notes_df.sample(n=min(10000, len(notes_df))))

In [8]:
# 第6步：分块合并数据（如果需要）
def merge_data_in_chunks():
    print("\nMerging data in chunks...")
    
    chunk_size = 50000  # 可以调整
    merged_chunks = []
    
    notes_df = pd.read_pickle(PROCESSED_PATH / 'processed_notes.pkl')
    patients_df = pd.read_pickle(PROCESSED_PATH / 'processed_patients.pkl')
    diagnoses_df = pd.read_pickle(PROCESSED_PATH / 'processed_diagnoses.pkl')
    
    for chunk in tqdm(np.array_split(notes_df, len(notes_df) // chunk_size + 1)):
        # 读取对应的处方数据
        prescriptions_chunk = pd.read_pickle(PROCESSED_PATH / 'processed_prescriptions.pkl')
        
        merged_chunk = preprocessor.merge_patient_data(
            chunk,
            prescriptions_chunk,
            diagnoses_df,
            patients_df
        )
        merged_chunks.append(merged_chunk)
        del chunk, prescriptions_chunk
        gc.collect()
    
    merged_df = pd.concat(merged_chunks, ignore_index=True)
    del merged_chunks, notes_df, prescriptions_df, diagnoses_df, patients_df
    gc.collect()
    
    return merged_df

In [ ]:
try:
    merged_df = merge_data_in_chunks()
except Exception as e:
    print(f"Error during analysis: {str(e)}")
    raise


Merging data in chunks...


  0%|                                                                              | 0/3 [00:00<?, ?it/s]2024-11-16 03:38:02,135 - INFO - Starting patient data merge...
2024-11-16 03:38:02,135 - INFO - Available columns before merge:
2024-11-16 03:38:02,136 - INFO - Notes columns: ['SUBJECT_ID', 'HADM_ID', 'cleaned_text', 'sections']
2024-11-16 03:38:02,136 - INFO - Prescriptions columns: ['SUBJECT_ID', 'HADM_ID', 'STARTDATE', 'ENDDATE', 'DRUG', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'ROUTE', 'DURATION', 'DRUG_NAME_GENERIC', 'drug_id']
2024-11-16 03:38:02,137 - INFO - Diagnoses columns: ['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']
2024-11-16 03:38:02,137 - INFO - Patients columns: ['SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG', 'AGE', 'AGE_GROUP', 'age_is_accurate', 'MORTALITY_STATUS']
2024-11-16 03:38:23,395 - INFO - Completed data merge, final dataset shape: (158405445, 21)
2024-11-16 03:38:23,396 - INFO - Final columns in merged dataset:
2024-11-16 03:38:23,397 - INFO

In [ ]:
try:
    print("\nCreating train-test split and saving...")
    train_df, test_df = preprocessor.create_train_test_split(merged_df)
except Exception as e:
    print(f"Error during analysis: {str(e)}")
    raise

In [ ]:
try:   
    # 分块保存
    chunk_size = 100000
    print("Saving training data in chunks...")
    for i, chunk in enumerate(np.array_split(train_df, len(train_df) // chunk_size + 1)):
        chunk.to_csv(PROCESSED_PATH / f'train_data_chunk_{i}.csv', index=False)

    print("Saving test data in chunks...")
    for i, chunk in enumerate(np.array_split(test_df, len(test_df) // chunk_size + 1)):
        chunk.to_csv(PROCESSED_PATH / f'test_data_chunk_{i}.csv', index=False)

    print("\nProcessing completed successfully!")
    print(f"Training set size: {len(train_df)}")
    print(f"Test set size: {len(test_df)}")
    print(f"Data saved to: {PROCESSED_PATH}")

except Exception as e:
    print(f"Error during processing: {str(e)}")
    raise